# Imports and Constants

In [15]:
import os
import shutil
import cv2
import numpy as np
import csv
import glob
import concurrent.futures
import tkinter as tk
import mediapipe as mp
from datetime import datetime
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from multiprocessing import Pool
from tkinter import filedialog, messagebox, Tk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn_extra.cluster import KMedoids
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean


# Configurable variables and constants
LOOK_AHEAD = 3
SENSITIVITY = 0.3
MULTIPLIER = 200
SEQUENCE_LENGTH = 30
NO_SEQUENCES = 30

actions = []
DATA_PATH = './actions'
CSV_FILE = 'action_directory.csv'
VIDEOS_PATH = './videos'
Model_Name = ""

# Mediapipe constants
NUM_LANDMARKS_HAND = 21 * 3  # Each hand has 21 landmarks with x, y, z coordinates

# Mediapipe models and drawing utils
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Miscellaneous Functions and Video Editing Functions

In [16]:
def extract_hand_landmarks(results):
    """Extract hand landmarks from Mediapipe results."""
    left_hand = [[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark] if results.left_hand_landmarks else []
    right_hand = [[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark] if results.right_hand_landmarks else []
    return left_hand + right_hand

def mediapipe_detection(image, model):
    """Perform Mediapipe detection on an image."""
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    # Keep image in RGB for drawing
    return image, results

def extract_keypoints(results):
    """Extract keypoints from Mediapipe results."""
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
    )
    # Draw pose connections
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
    )
    # Draw left hand connections
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
    )
    # Draw right hand connections
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
    )

# Refine MP Data

In [17]:
import os
import numpy as np
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from sklearn_extra.cluster import KMedoids
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Constants
NUM_LANDMARKS_HAND = 21 * 3  # Each hand has 21 landmarks with x, y, z coordinates

def data_refine(action_data_path):
    sequences = load_sequences(action_data_path)
    
    # NEW: Remove frames without hands
    print("\nRemoving frames without visible hands...")
    sequences = remove_frames_without_hands(sequences)
    
    # Prepare reference features
    if not sequences:
        print("No sequences with visible hands found. Exiting.")
        return
    reference_sequence = sequences[0]  # Choose a sequence that contains the main pattern
    reference_features = extract_features(reference_sequence)
    threshold = 0.8  # Adjust this threshold as needed
    
    # Process sequences to create new data
    new_sequences = []
    
    # First 10 sequences: Isolate pattern and create sequences with slight variations
    pattern_sequences = create_pattern_sequences(sequences, reference_features, threshold, num_sequences=10)
    new_sequences.extend(pattern_sequences)
    
    # Next 10 sequences: Trim existing sequences to 30 frames keeping main movement
    trimmed_sequences = trim_sequences_to_pattern(sequences, reference_features, threshold, num_sequences=10)
    new_sequences.extend(trimmed_sequences)
    
    # Last 10 sequences: Reduce sequences by removing frames until they are 30 frames long
    reduced_sequences = reduce_sequences(sequences, reference_features, threshold, num_sequences=10)
    new_sequences.extend(reduced_sequences)
    
    # Ensure we have exactly 30 sequences
    new_sequences = new_sequences[:30]
    
    # Save new sequences
    save_sequences(new_sequences, action_data_path)
    
    # Interpolation and Filtering
    print("\nStarting interpolation and filtering...")
    interpolate_and_filter_sequences(new_sequences, action_data_path)
    
    # NEW: Ensure all sequences have 30 frames
    print("\nEnsuring all sequences have 30 frames...")
    sequences = load_sequences(action_data_path)  # Load sequences again after interpolation and filtering
    sequences = pad_sequences_to_length(sequences, length=30)
    # Save the padded sequences
    save_sequences(sequences, action_data_path)
    print("All sequences have been padded to 30 frames if necessary.")

def load_sequences(action_data_path):
    sequences = []
    seq_dirs = [d for d in os.listdir(action_data_path) if os.path.isdir(os.path.join(action_data_path, d))]
    seq_dirs = sorted(seq_dirs, key=lambda x: int(x.split('_')[1]))  # Ensure consistent order
    for seq_dir in seq_dirs:
        seq_path = os.path.join(action_data_path, seq_dir)
        frames = []
        frame_files = sorted([f for f in os.listdir(seq_path) if f.endswith('.npy')],
                             key=lambda x: int(x.split('_')[1].split('.')[0]))
        for frame_file in frame_files:
            frame_data = np.load(os.path.join(seq_path, frame_file))
            frames.append(frame_data)
        sequences.append(frames)
    return sequences

def save_sequences(sequences, action_data_path):
    # Remove existing data
    shutil.rmtree(action_data_path)
    os.makedirs(action_data_path, exist_ok=True)
    for i, seq in enumerate(sequences):
        seq_dir = os.path.join(action_data_path, f'seq_{i}')
        os.makedirs(seq_dir, exist_ok=True)
        for j, frame in enumerate(seq):
            frame_file = os.path.join(seq_dir, f'frame_{j}.npy')
            if frame is not None:
                np.save(frame_file, frame)
    print(f"Sequences saved to {action_data_path}")

def create_pattern_sequences(sequences, reference_features, threshold, num_sequences):
    new_sequences = []
    for _ in range(num_sequences):
        seq = select_sequence_with_pattern(sequences, reference_features, threshold)
        if seq is None:
            break
        seq_variation = add_variation(seq)
        seq_trimmed = trim_or_extend_sequence(seq_variation, length=30)
        new_sequences.append(seq_trimmed)
    return new_sequences

def trim_sequences_to_pattern(sequences, reference_features, threshold, num_sequences):
    new_sequences = []
    for _ in range(num_sequences):
        seq = select_sequence_with_pattern(sequences, reference_features, threshold)
        if seq is None:
            break
        # NEW: Trim sequence to where hands are visible
        seq_trimmed = trim_sequence_to_hands_visible(seq, length=30)
        new_sequences.append(seq_trimmed)
    return new_sequences

def reduce_sequences(sequences, reference_features, threshold, num_sequences):
    new_sequences = []
    for _ in range(num_sequences):
        seq = select_sequence_with_pattern(sequences, reference_features, threshold)
        if seq is None:
            break
        # NEW: Remove frames without hands before reducing
        seq = remove_frames_without_hands_in_sequence(seq)
        seq_reduced = remove_frames_to_length(seq, length=30)
        new_sequences.append(seq_reduced)
    return new_sequences

def select_sequence_with_pattern(sequences, reference_features, threshold):
    for seq in sequences:
        if contains_main_pattern(seq, reference_features, threshold):
            return seq
    return None

def contains_main_pattern(sequence, reference_features, threshold):
    if not sequence:
        return False
    sequence_features = extract_features(sequence)
    similarity = compute_similarity(sequence_features, reference_features)
    return similarity >= threshold

def extract_features(sequence):
    normalized_sequence = normalize_sequence(sequence)
    features = np.array([frame.flatten() for frame in normalized_sequence]).flatten()
    return features

def normalize_sequence(sequence):
    return [normalize_frame(frame) for frame in sequence]

def normalize_frame(frame):
    # Determine the number of keypoints
    num_keypoints = frame.shape[0] // 3
    # Reshape the frame into (num_keypoints, 3)
    frame_reshaped = frame.reshape((num_keypoints, 3))
    # Use the first keypoint as the reference point
    reference_point = frame_reshaped[0]  # Adjust index as per your data
    # Translate keypoints so that reference point is at the origin
    translated_frame = frame_reshaped - reference_point
    # Compute distances from the origin for scaling
    distances = np.linalg.norm(translated_frame, axis=1)
    max_distance = np.max(distances)
    # Scale the frame if max_distance is greater than zero
    if max_distance > 0:
        scaled_frame = translated_frame / max_distance
    else:
        scaled_frame = translated_frame
    # Flatten the scaled frame back to 1D
    return scaled_frame.flatten()

def compute_similarity(features1, features2):
    min_length = min(len(features1), len(features2))
    features1 = features1[:min_length].reshape(1, -1)
    features2 = features2[:min_length].reshape(1, -1)
    similarity = cosine_similarity(features1, features2)[0][0]
    return similarity

def add_variation(sequence):
    return [frame + np.random.normal(0, 0.01, frame.shape) for frame in sequence]

def trim_or_extend_sequence(sequence, length):
    if len(sequence) > length:
        # NEW: Trim frames where hands are not visible
        sequence = remove_frames_without_hands_in_sequence(sequence)
        if len(sequence) > length:
            start = (len(sequence) - length) // 2
            return sequence[start:start+length]
    # Extend sequence if needed
    sequence_extended = sequence.copy()
    while len(sequence_extended) < length:
        sequence_extended.append(sequence_extended[-1])
    return sequence_extended[:length]

def trim_sequence_to_main_movement(sequence, reference_features, length):
    max_similarity = -1
    best_start = 0
    # NEW: Remove frames without hands before trimming
    sequence = remove_frames_without_hands_in_sequence(sequence)
    for start in range(0, len(sequence) - length + 1):
        window_sequence = sequence[start:start+length]
        window_features = extract_features(window_sequence)
        similarity = compute_similarity(window_features, reference_features)
        if similarity > max_similarity:
            max_similarity = similarity
            best_start = start
    return sequence[best_start:best_start+length]

# NEW FUNCTION: Trim sequence to where hands are visible and adjust length
def trim_sequence_to_hands_visible(sequence, length):
    # Remove frames without hands
    sequence = remove_frames_without_hands_in_sequence(sequence)
    # If sequence is longer than desired length, trim equally from both ends
    if len(sequence) > length:
        extra_frames = len(sequence) - length
        start_trim = extra_frames // 2
        end_trim = extra_frames - start_trim
        sequence = sequence[start_trim: len(sequence) - end_trim]
    # If sequence is shorter, pad it
    elif len(sequence) < length:
        sequence = pad_sequence(sequence, length)
    return sequence

# Helper function to pad a sequence
def pad_sequence(sequence, length):
    sequence_extended = sequence.copy()
    while len(sequence_extended) < length:
        sequence_extended.append(sequence_extended[-1])
    return sequence_extended[:length]

def remove_frames_to_length(sequence, length):
    if len(sequence) <= length:
        return sequence
    indices = np.linspace(0, len(sequence) - 1, num=length, dtype=int)
    return [sequence[i] for i in indices]

def interpolate_and_filter_sequences(sequences, action_data_path):
    # Interpolate missing keypoints in each sequence
    for idx, sequence in enumerate(sequences):
        print(f"Interpolating missing data in sequence {idx}...")
        sequences[idx] = interpolate_sequence(sequence)
    
    # Find the representative sequence
    try:
        print("\nFinding representative sequence...")
        flattened_sequences = [np.array(seq).flatten() for seq in sequences if seq]
        if not flattened_sequences:
            print("No sequences available for finding representative sequence.")
            return
        X = np.array(flattened_sequences)
        kmedoids = KMedoids(n_clusters=1, metric='euclidean', random_state=0).fit(X)
        medoid_index = kmedoids.medoid_indices_[0]
        representative_sequence = sequences[medoid_index]
    except Exception as e:
        print(f"Error finding representative sequence: {e}")
        representative_sequence = sequences[0]
    
    # Filter each sequence
    for idx, seq in enumerate(sequences):
        print(f"\nFiltering sequence {idx}...")
        try:
            if seq:
                filtered_seq = filter_sequence(seq, representative_sequence)
                sequences[idx] = filtered_seq
            else:
                print(f"Sequence {idx} is empty after interpolation. Skipping filtering.")
        except Exception as e:
            print(f"Error filtering sequence {idx}: {e}")
            continue
    
    # Save the interpolated and filtered sequences
    save_sequences(sequences, action_data_path)
    print("\nInterpolation and filtering completed.")

def interpolate_sequence(sequence):
    try:
        lh_start = 33 * 4 + 468 * 3
        lh_end = lh_start + NUM_LANDMARKS_HAND
        rh_start = lh_end
        rh_end = rh_start + NUM_LANDMARKS_HAND

        for i in range(1, len(sequence) - 1):
            frame = sequence[i]
            # Left hand keypoints
            for idx in range(lh_start, lh_end, 3):
                if np.all(frame[idx:idx + 3] == 0):
                    prev_idx, next_idx = find_valid_indices(sequence, i, idx)
                    if prev_idx is not None and next_idx is not None:
                        interpolate_frames(sequence, prev_idx, next_idx, idx)
            # Right hand keypoints
            for idx in range(rh_start, rh_end, 3):
                if np.all(frame[idx:idx + 3] == 0):
                    prev_idx, next_idx = find_valid_indices(sequence, i, idx)
                    if prev_idx is not None and next_idx is not None:
                        interpolate_frames(sequence, prev_idx, next_idx, idx)
        return sequence
    except Exception as e:
        print(f"Error during interpolation: {e}")
        return sequence

def find_valid_indices(sequence, current_index, idx):
    prev_idx = next_idx = None
    for j in range(current_index - 1, -1, -1):
        if not np.all(sequence[j][idx:idx + 3] == 0):
            prev_idx = j
            break
    for j in range(current_index + 1, len(sequence)):
        if not np.all(sequence[j][idx:idx + 3] == 0):
            next_idx = j
            break
    return prev_idx, next_idx

def interpolate_frames(sequence, prev_idx, next_idx, idx):
    for interp_idx in range(prev_idx + 1, next_idx):
        alpha = (interp_idx - prev_idx) / (next_idx - prev_idx)
        sequence[interp_idx][idx:idx + 3] = (1 - alpha) * sequence[prev_idx][idx:idx + 3] + alpha * sequence[next_idx][idx:idx + 3]

def filter_sequence(seq, representative_sequence):
    try:
        distance, path = fastdtw(seq, representative_sequence, dist=euclidean)
        indices_seq = [index[0] for index in path]
        segments = []
        start = indices_seq[0]
        prev = indices_seq[0]
        for idx_seq in indices_seq[1:]:
            if idx_seq == prev + 1:
                prev = idx_seq
            else:
                segments.append((start, prev))
                start = idx_seq
                prev = idx_seq
        segments.append((start, prev))
        longest_segment = max(segments, key=lambda x: x[1] - x[0])
        start_idx, end_idx = longest_segment
        filtered_seq = [None if i < start_idx or i > end_idx else seq[i] for i in range(len(seq))]
        print(f"Filtered sequence length: {len([f for f in filtered_seq if f is not None])} frames (non-matching frames removed)")
        return filtered_seq
    except Exception as e:
        print(f"Error during filtering: {e}")
        return seq

# NEW FUNCTION: Pad sequences to a specified length
def pad_sequences_to_length(sequences, length=30):
    for idx, seq in enumerate(sequences):
        # Remove None frames that may have been introduced
        seq = [frame for frame in seq if frame is not None]
        sequences[idx] = seq
        actual_length = len(seq)
        if actual_length < length:
            frames_to_add = length - actual_length
            if actual_length > 0:
                last_frame = seq[-1]
                seq.extend([last_frame.copy() for _ in range(frames_to_add)])
                print(f"Sequence {idx} padded with {frames_to_add} frame(s).")
            else:
                print(f"Sequence {idx} is empty. Cannot pad an empty sequence.")
        elif actual_length > length:
            # Trim the sequence to the desired length
            sequences[idx] = seq[:length]
    return sequences

# NEW FUNCTION: Remove frames without visible hands from all sequences
def remove_frames_without_hands(sequences):
    for idx, seq in enumerate(sequences):
        sequences[idx] = remove_frames_without_hands_in_sequence(seq)
    # Remove empty sequences
    sequences = [seq for seq in sequences if seq]
    return sequences

# NEW FUNCTION: Remove frames without visible hands in a single sequence
def remove_frames_without_hands_in_sequence(sequence):
    lh_start = 33 * 4 + 468 * 3
    lh_end = lh_start + NUM_LANDMARKS_HAND
    rh_start = lh_end
    rh_end = rh_start + NUM_LANDMARKS_HAND
    new_sequence = []
    for frame in sequence:
        left_hand = frame[lh_start:lh_end]
        right_hand = frame[rh_start:rh_end]
        # Check if hands are visible (i.e., not all zeros)
        if not (np.all(left_hand == 0) and np.all(right_hand == 0)):
            new_sequence.append(frame)
    return new_sequence

# Example usage:
# action_data_path = '/path/to/action_data'  # Replace with your actual path
# data_refine(action_data_path)


# Extract Movement Data from Videos

In [18]:
def full_MP_Extraction(video_path, output_directory):
    print(video_path)
    print(output_directory)
    video_path = "./videos/"+video_path
    """
    Extract keypoints from each frame of a video and save as .npy files.

    Args:
        video_path (str): Path to the input video file.
        output_directory (str): Directory to save the output .npy files.
        model: Mediapipe model used for detecting keypoints (e.g., Hands, Holistic).
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # Exit if video ends

            # Perform Mediapipe detection on the frame
            image, results = mediapipe_detection(frame, holistic)

            # Extract keypoints from the frame
            keypoints = extract_keypoints(results)

            # Define the output filename based on frame number
            output_filename = f"frame_{frame_count}.npy"
            output_path = os.path.join(output_directory, output_filename)

            # Save keypoints as a .npy file
            np.save(output_path, keypoints)

            frame_count += 1

# Create and Save Model

In [19]:
# Function to create and save the model
def create_and_save_model(DATA_PATH= ""):
    if DATA_PATH == "":
        root = Tk()
        root.withdraw()  # Hide the root window
        DATA_PATH = filedialog.askdirectory(title="Select the file in Model Data you want to create")
        
    model_name = os.path.basename(DATA_PATH)  # Use the folder name as the model name
    actions_csv_path = os.path.join(DATA_PATH, 'actions.csv')

    if not os.path.exists(actions_csv_path):
        print(f"Error: actions.csv not found in {DATA_PATH}")
        return

    # Load actions from the CSV file (assuming actions are in the second column of the CSV)
    actions = []
    with open(actions_csv_path, mode='r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            actions.append(row[1])  # Assuming the action is in the second column

    # Create a label map from the loaded actions
    label_map = {label: num for num, label in enumerate(actions)}
    sequence_length = 30  # Set your sequence length

    sequences, labels = [], []
    for action in actions:
        action_dir = os.path.join(DATA_PATH + "/actions", action)
        if os.path.exists(action_dir) and os.path.isdir(action_dir):
            sequence_dirs = [d for d in os.listdir(action_dir) if d.startswith('seq_')]

            action_sequences = []
            for sequence_dir in sequence_dirs:
                window = []
                for frame_num in range(sequence_length):
                    npy_path = os.path.join(action_dir, sequence_dir, f"frame_{frame_num}.npy")
                    if os.path.exists(npy_path):  # Ensure the file exists
                        res = np.load(npy_path)
                        window.append(res)
                    else:
                        print(f"Frame {npy_path} not found, skipping.")
                if len(window) == sequence_length:  # Ensure the full sequence length is met
                    action_sequences.append(window)

            if len(action_sequences) == 0:
                print(f"No valid sequences found for action: {action}")
                continue

            # Add the sequences to the dataset
            sequences.extend(action_sequences)
            labels.extend([label_map[action]] * len(action_sequences))

    # Ensure sequences and labels are not empty
    if len(sequences) == 0 or len(labels) == 0:
        print("No valid sequences or labels found. Exiting.")
        return

    # Convert the lists to numpy arrays
    X = np.array(sequences)
    y = to_categorical(labels).astype(int)

    print(f"X shape: {X.shape}, y shape: {y.shape}")

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

    log_dir = os.path.join('Logs')
    tb_callback = TensorBoard(log_dir=log_dir)

    # Define and compile the model
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, X.shape[2])))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(actions), activation='softmax'))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
    model.summary()

    # Create the Models directory if it doesn't exist
    model_dir = DATA_PATH
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    # Save the model and its weights
    model_path = os.path.join(model_dir, f'{model_name}.h5')
    weights_path = os.path.join(model_dir, f'{model_name}_weights.h5')

    model.save(model_path)  # Save the entire model
    model.save_weights(weights_path)  # Save the model weights separately

    print(f"Model saved at: {model_path}")
    print(f"Model weights saved at: {weights_path}")
    interpretation(DATA_PATH)

# Call the function to create and save the model
#create_and_save_model()


# Live Interpretation

In [20]:
import os
import numpy as np
import cv2
import mediapipe as mp
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog, messagebox
import csv

def mediapipe_detection(image, model):
    """Process an image and return the results."""
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert color space
    image.flags.writeable = False  # Improve performance
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Set back to writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert back to BGR
    return image, results

def draw_styled_landmarks(image, results):
    """Draw landmarks on the image."""
    mp_drawing = mp.solutions.drawing_utils
    # Draw face connections
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp.solutions.holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    )
    # Draw pose connections
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp.solutions.holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    # Draw left hand connections
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    )
    # Draw right hand connections
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    )

def extract_keypoints(results):
    """Extract keypoints from Mediapipe detection results."""
    # Extract pose landmarks
    pose = np.array(
        [[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]
    ).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    # Extract face landmarks
    face = np.array(
        [[res.x, res.y, res.z] for res in results.face_landmarks.landmark]
    ).flatten() if results.face_landmarks else np.zeros(468 * 3)
    # Extract left hand landmarks
    left_hand = np.array(
        [[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]
    ).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    # Extract right hand landmarks
    right_hand = np.array(
        [[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]
    ).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    # Concatenate all landmarks
    keypoints = np.concatenate([pose, face, left_hand, right_hand])
    return keypoints

def normalize_frame(frame):
    """Normalize a single frame of keypoints."""
    num_keypoints = frame.shape[0] // 3
    frame_reshaped = frame.reshape((num_keypoints, 3))
    reference_point = frame_reshaped[0]  # Adjust index as per your data
    translated_frame = frame_reshaped - reference_point
    distances = np.linalg.norm(translated_frame, axis=1)
    max_distance = np.max(distances)
    if max_distance > 0:
        scaled_frame = translated_frame / max_distance
    else:
        scaled_frame = translated_frame
    return scaled_frame.flatten()

def hands_are_visible(frame):
    """Check if hands are visible in the frame."""
    lh_start = 33 * 4 + 468 * 3
    lh_end = lh_start + 21 * 3
    rh_start = lh_end
    rh_end = rh_start + 21 * 3
    left_hand = frame[lh_start:lh_end]
    right_hand = frame[rh_start:rh_end]
    return not (np.all(left_hand == 0) and np.all(right_hand == 0))

def interpretation(DATA_PATH = ""):
    if DATA_PATH == "":
        root = tk.Tk()
        root.withdraw()  # Hide the main Tkinter window
        DATA_PATH = filedialog.askdirectory(title="Select directory containing actions.csv and model file (.h5)")

    # Paths for actions.csv and model file
    model_name = os.path.basename(DATA_PATH)  # Use the folder name as the model name
    actions_csv_path = os.path.join(DATA_PATH, 'actions.csv')

    if not os.path.exists(actions_csv_path):
        print(f"Error: actions.csv not found in {DATA_PATH}")
        return

    # Load actions from actions.csv
    actions = []
    with open(actions_csv_path, mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            actions.append(row[1])  # Adjust index if action names are in a different column

    actions = np.array(actions)

    # Verify actions
    print("Actions:", actions)
    print("Number of actions:", len(actions))

    if len(actions) == 0:
        print("No actions found in actions.csv.")
        return

    # Look for the model file (.h5) in the directory
    model_path = None
    for file in os.listdir(DATA_PATH):
        if file.endswith(".h5"):
            model_path = os.path.join(DATA_PATH, file)
            break

    if not model_path:
        messagebox.showerror("Error", "No model file (.h5) found in the selected directory.")
        return

    # Load pre-trained model
    model = tf.keras.models.load_model(model_path)
    model.summary()

    # Initialize variables
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.5  # Adjust the threshold as needed
    sequence_length = 30  # Ensure this matches your model's expected input length
    cap = cv2.VideoCapture(0)

    # Set up Mediapipe model
    mp_holistic = mp.solutions.holistic
    with mp_holistic.Holistic(min_detection_confidence=0.5, 
                              min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            # Read video feed
            ret, frame = cap.read()
            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # Extract keypoints
            keypoints = extract_keypoints(results)
            
            # Check if hands are visible
            if hands_are_visible(keypoints):
                # Normalize the keypoints
                keypoints_normalized = normalize_frame(keypoints)
                # Update the sequence
                sequence.append(keypoints_normalized)
                sequence = sequence[-sequence_length:]  # Ensure sequence length

                if len(sequence) == sequence_length:
                    # Make a prediction
                    res = model.predict(np.expand_dims(sequence, axis=0))[0]
                    predictions.append(np.argmax(res))
                    
                    # Visualization logic: Only update sentence if prediction is stable
                    if np.unique(predictions[-10:])[0] == np.argmax(res):
                        if res[np.argmax(res)] > threshold:
                            if len(sentence) > 0:
                                if actions[np.argmax(res)] != sentence[-1]:
                                    sentence.append(actions[np.argmax(res)])
                            else:
                                sentence.append(actions[np.argmax(res)])

                        # Keep the sentence length limited to 5
                        if len(sentence) > 5:
                            sentence = sentence[-5:]
            else:
                # Optionally, reset the sequence if hands are not visible
                sequence = []
                predictions = []

            # Display predictions on the screen
            cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Show the frame
            cv2.imshow('Realtime LSTM Sign Language Detection', image)

            # Break the loop gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    # Release video capture and close windows
    cap.release()
    cv2.destroyAllWindows()





# GUI Workflow

In [21]:
import tkinter as tk
from tkinter import messagebox, filedialog
import os
import random
import csv

# Global variables
new_csv_path = ""
is_random = False
num_actions = 0
specific_action_lines = []
actions = []
open_window = None  # Keep track of the currently open window

# Load actions from the CSV file
def load_actions():
    global actions
    global CSV_FILE
    if not os.path.exists(CSV_FILE):
        print(f"CSV_FILE '{CSV_FILE}' does not exist.")
        return
    with open(CSV_FILE, mode='r') as file:
        reader = csv.reader(file)
        actions = [row for row in reader]

# Close the current window if one is open
def close_current_window():
    global open_window
    if open_window is not None:
        open_window.destroy()
        open_window = None

# First window to create a new model directory
def create_new_model():
    def on_create_model():
        model_name = "Model_data/" + model_name_entry.get()
        global DATA_PATH
        DATA_PATH = model_name

        if not model_name_entry.get():
            messagebox.showerror("Error", "Model name cannot be empty!")
            return
        model_dir = os.path.join(os.getcwd(), model_name)

        if os.path.exists(model_dir):
            messagebox.showerror("Error", "Directory already exists!")
        else:
            os.makedirs(model_dir)
            messagebox.showinfo("Success", f"Directory '{model_name}' created!")
            open_create_from_window(model_name, model_window)  # Pass model_window to close later

    # Close the current window before opening a new one
    close_current_window()

    # GUI for model creation
    model_window = tk.Toplevel(root)
    model_window.title("Create New Model")
    model_window.geometry("400x200")
    
    # Track the newly opened window
    global open_window
    open_window = model_window

    model_name_label = tk.Label(model_window, text="Enter new model name:")
    model_name_label.pack(pady=10)

    model_name_entry = tk.Entry(model_window)
    model_name_entry.pack(pady=10)

    create_model_button = tk.Button(model_window, text="Create Model", command=on_create_model)
    create_model_button.pack(pady=10)

# New window "Create From..."
def open_create_from_window(model_name, previous_window):
    def open_csv_creation():
        previous_window.destroy()  # Close previous window
        create_action_selection_window(model_name, create_from_window)

    def open_existing_videos():
        previous_window.destroy()  # Close previous window
        from_videos()  # Open the from_videos window

    def open_existing_mp_data():
        previous_window.destroy()  # Close previous window
        from_mp_data()  # Handle the existing MP Data option

    # Close the current window before opening a new one
    close_current_window()

    create_from_window = tk.Toplevel(root)
    create_from_window.title("Create From...")
    create_from_window.geometry("400x300")

    # Track the newly opened window
    global open_window
    open_window = create_from_window

    # Label
    create_from_label = tk.Label(create_from_window, text="How would you like to create new model?")
    create_from_label.pack(pady=10)

    # Fresh Button
    fresh_button = tk.Button(create_from_window, text="Fresh", command=open_csv_creation)
    fresh_button.pack(pady=10)
    tk.Label(create_from_window, text="Create model by selecting actions, preprocess and process videos, extract MP data, test, train, and save model.").pack(pady=5)

    # Existing Videos Button
    existing_videos_button = tk.Button(create_from_window, text="Existing MP Data", command=open_existing_videos)
    existing_videos_button.pack(pady=10)
    tk.Label(create_from_window, text="Create models from existing MP data, test, train, and save model.").pack(pady=5)



# Function to process videos
def from_videos(processed=True):
    def begin_processing_and_extraction():
        model_action_path = os.path.join(DATA_PATH, "actions")
        os.makedirs(model_action_path, exist_ok=True)

        if not processed:
            # Process actions based on CSV_FILE
            for line in actions:
                action_name_dir = os.path.join(model_action_path, line[1])
                os.makedirs(action_name_dir, exist_ok=True)
                i = 0
                for element in line[2:]:  # Skipping pos 0 and pos 1 elements
                    mp_path = os.path.join(action_name_dir, "seq_"+str(i))
                    full_MP_Extraction(element, mp_path)
                    i += 1
                data_refine(action_name_dir)
            messagebox.showinfo("Success", "Processing and extraction complete!")
        else:
            # Open file explorer for user to select directory
            selected_path = filedialog.askdirectory()
            if selected_path:
                actions_dir = os.path.join(selected_path, 'actions')
                actions_csv = os.path.join(selected_path, 'actions.csv')
                if os.path.exists(actions_dir) and os.path.exists(actions_csv):
                    # Copy the 'actions' directory and 'actions.csv' into DATA_PATH
                    target_actions_dir = os.path.join(DATA_PATH, 'actions')
                    target_actions_csv = os.path.join(DATA_PATH, 'actions.csv')
                    os.makedirs(target_actions_dir, exist_ok=True)

                    # Copy actions directory and actions.csv
                    os.system(f'cp -r "{actions_dir}" "{target_actions_dir}"')
                    os.system(f'cp "{actions_csv}" "{target_actions_csv}"')
                    
                    messagebox.showinfo("Success", "Preprocessed data has been copied!")
                else:
                    messagebox.showerror("Error", "Selected directory must contain 'actions' and 'actions.csv'.")
            else:
                messagebox.showerror("Error", "No directory selected.")
        create_and_save_model(DATA_PATH)

    # Close the current window before opening a new one
    close_current_window()

    video_window = tk.Toplevel(root)
    video_window.title("From Videos")
    video_window.geometry("400x300")

    # Track the newly opened window
    global open_window
    open_window = video_window

    # Instruction label
    if not processed:
        instruction_label = tk.Label(video_window, text="Processing unprocessed data from CSV.")
    else:
        instruction_label = tk.Label(video_window, text="Select preprocessed data directory.")
    instruction_label.pack(pady=10)

    # Add a button to start processing
    if processed:
        select_data_button = tk.Button(video_window, text="Select Data Directory", command=begin_processing_and_extraction)
        select_data_button.pack(pady=20)
    else:
        process_button = tk.Button(video_window, text="Begin Processing and Extraction", command=begin_processing_and_extraction)
        process_button.pack(pady=20)

# Process MP data (placeholder)
def from_mp_data():
    close_current_window()
    mp_window = tk.Toplevel(root)
    mp_window.title("From MP Data")
    mp_window.geometry("400x300")

    global open_window
    open_window = mp_window

    tk.Label(mp_window, text="This is the MP Data window.").pack()

# Second window to select actions for creating CSV
def create_action_selection_window(model_name, previous_window):
    def on_checkbox_selected(selected_var):
        # Deselect other checkboxes when one is selected
        if selected_var == all_actions_var:
            number_of_actions_var.set(0)
            specify_actions_var.set(0)
            number_of_actions_frame.pack_forget()
            specify_actions_frame.pack_forget()
        elif selected_var == number_of_actions_var:
            all_actions_var.set(0)
            specify_actions_var.set(0)
            number_of_actions_frame.pack(pady=10)
            specify_actions_frame.pack_forget()
        elif selected_var == specify_actions_var:
            all_actions_var.set(0)
            number_of_actions_var.set(0)
            number_of_actions_frame.pack_forget()
            specify_actions_frame.pack(pady=10)

    def on_create_model_csv():
        global is_random
        new_csv_path = os.path.join(os.getcwd(), model_name, "actions.csv")
        with open(new_csv_path, 'w', newline='') as new_csv:
            writer = csv.writer(new_csv)
            if all_actions_var.get():
                # Copy all actions from the CSV file
                for row in actions:
                    writer.writerow(row)
            elif number_of_actions_var.get():
                # Select a number of actions
                num = int(num_actions_entry.get())
                is_random = random_checkbox_var.get()
                if num > len(actions):
                    messagebox.showerror("Error", f"Number exceeds total actions ({len(actions)}).")
                    return
                selected_actions = random.sample(actions, num) if is_random else actions[:num]
                for row in selected_actions:
                    writer.writerow(row)
            elif specify_actions_var.get():
                # Write specific actions to new CSV
                if not specific_action_lines:
                    messagebox.showerror("Error", "No actions selected.")
                    return
                for line in specific_action_lines:
                    writer.writerow(line)
            else:
                messagebox.showerror("Error", "No option selected.")
                return

        messagebox.showinfo("Success", f"CSV created at: {new_csv_path}")
        previous_window.destroy()  # Close the CSV selection window
        global CSV_FILE
        CSV_FILE = new_csv_path
        load_actions()
        from_videos(False)  # Call `from_videos` after creating CSV

    # Close the current window before opening a new one
    close_current_window()

    action_window = tk.Toplevel(root)
    previous_window.destroy()  # Close the "Create From" window
    action_window.title("Select Actions")
    action_window.geometry("600x600")

    global open_window
    open_window = action_window

    # --- Action Selection Checkboxes ---
    all_actions_var = tk.IntVar(value=1)
    number_of_actions_var = tk.IntVar(value=0)
    specify_actions_var = tk.IntVar(value=0)

    all_actions_checkbox = tk.Checkbutton(action_window, text="All Actions", variable=all_actions_var, command=lambda: on_checkbox_selected(all_actions_var))
    all_actions_checkbox.pack(anchor='w')

    number_of_actions_checkbox = tk.Checkbutton(action_window, text="# of Actions", variable=number_of_actions_var, command=lambda: on_checkbox_selected(number_of_actions_var))
    number_of_actions_checkbox.pack(anchor='w')

    specify_actions_checkbox = tk.Checkbutton(action_window, text="Specify Actions", variable=specify_actions_var, command=lambda: on_checkbox_selected(specify_actions_var))
    specify_actions_checkbox.pack(anchor='w')

    # --- Number of Actions Section ---
    number_of_actions_frame = tk.Frame(action_window)
    tk.Label(number_of_actions_frame, text="Number of Actions:").pack(side=tk.LEFT)
    num_actions_entry = tk.Entry(number_of_actions_frame)
    num_actions_entry.pack(side=tk.LEFT, padx=5)
    random_checkbox_var = tk.IntVar()
    random_checkbox = tk.Checkbutton(number_of_actions_frame, text="Random", variable=random_checkbox_var)
    random_checkbox.pack(side=tk.LEFT)

    # --- Specify Actions Section ---
    specify_actions_frame = tk.Frame(action_window)

    tk.Label(specify_actions_frame, text="Search Actions:").pack()
    search_entry_var = tk.StringVar()
    search_entry = tk.Entry(specify_actions_frame, textvariable=search_entry_var)
    search_entry.pack(pady=5)

    search_results_listbox = tk.Listbox(specify_actions_frame, selectmode=tk.MULTIPLE)
    search_results_listbox.pack(pady=5)

    tk.Label(specify_actions_frame, text="Selected Actions:").pack()
    selected_actions_listbox = tk.Listbox(specify_actions_frame)
    selected_actions_listbox.pack(pady=5)

    # Create model CSV button
    create_csv_button = tk.Button(action_window, text="Create CSV", command=on_create_model_csv)
    create_csv_button.pack(pady=20)

    # Initially display the appropriate frame based on checkbox selection
    on_checkbox_selected(all_actions_var)

# Function for opening the Settings window
def open_settings():
    # Close the current window before opening a new one
    close_current_window()

    # Create a new window for settings
    settings_window = tk.Toplevel(root)
    settings_window.title("Settings")
    settings_window.geometry("300x200")

    # Track the newly opened window
    global open_window
    open_window = settings_window

    tk.Label(settings_window, text="Settings Window").pack()

# Main window setup
root = tk.Tk()
root.title("Main Window")
root.geometry("300x150")  # Set the size of the main window

# Add the "Run Model" button (you'll need to implement `interpretation` function if needed)
run_model_button = tk.Button(root, text="Run Model", command=lambda: print("Run model pressed"))
run_model_button.pack(pady=20)

# Add the "Create Model" button to open model creation
create_model_button = tk.Button(root, text="Create Model", command=create_new_model)
create_model_button.pack(pady=20)

# Add the "Settings" button
settings_button = tk.Button(root, text="Settings", command=open_settings)
settings_button.pack(pady=10)

# Load actions from CSV when the program starts
load_actions()

# Run the main loop to display the window
root.mainloop()



28288.mp4
Model_data/test_8\actions\human\seq_0


c:\Users\Cal\miniconda3\envs\cs456\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


28291.mp4
Model_data/test_8\actions\human\seq_1
28292.mp4
Model_data/test_8\actions\human\seq_2
28296.mp4
Model_data/test_8\actions\human\seq_3

Removing frames without visible hands...
Sequences saved to Model_data/test_8\actions\human

Starting interpolation and filtering...
Interpolating missing data in sequence 0...
Interpolating missing data in sequence 1...
Interpolating missing data in sequence 2...
Interpolating missing data in sequence 3...
Interpolating missing data in sequence 4...
Interpolating missing data in sequence 5...
Interpolating missing data in sequence 6...
Interpolating missing data in sequence 7...
Interpolating missing data in sequence 8...
Interpolating missing data in sequence 9...
Interpolating missing data in sequence 10...
Interpolating missing data in sequence 11...
Interpolating missing data in sequence 12...
Interpolating missing data in sequence 13...
Interpolating missing data in sequence 14...
Interpolating missing data in sequence 15...
Interpolatin